# Select an objective

## Load data
First load a dataset containing some toy model predictions. Here we'll compare NNDAR against streamgage observations (obs). By nature, NNDAR predictions are 'out of sample'; otherwise, we'd need to use cross validation or information criteria to estimate the out-of-sample performance.

In [ ]:
# load data from s3 (run once)
import fsspec
import numpy as np
import pandas as pd

fs_read = fsspec.filesystem(
    "s3", anon=True, client_kwargs={"endpoint_url": "https://renc.osn.xsede.org"}
)

with fs_read.open("s3://rsignellbucket2/hytest/thodson/gages2_nndar.parquet") as f:
    df = pd.read_parquet(f)

# threshold low values
df[df < 0.01] = 0.01

## Overview
The first step in model evaluation is to select an appropriate model for representing error (or uncertainty) in your model;
in other words, select an objective function.

For example, if the errors are normally distributed and /iid/, 
the optimal objective function is mean squared error. 
We'll review why shortly. 
However, for many models, the error distribution is neither normal nor iid.
Runoff models being a pertinent example.
Their errors tend to scale with the magnitude of flow (heteroscedasticity).
Logging the data helps to mitigate that,
which is what most hydrologists want,
even if they don't do it in practice.


The next sections demonstrate a basic procedure for selecting a reasonable objective function for your model.

## Benchmarking objectives
Pretend that a model has a MSE of 1 and a MSLE of 1.
Which is a better score?
The short answer is we don't know.
These scores have different scales, so we can't compare them directly.


Fortunately, we already have everything we need.
The trick is to transform the objectives into likelihoods,
putting them all on a common scale.
We'll demonstrate the simplest approach of using maximum likelihood estimators.

### Log likelihoods
The first, and arguably de facto, objective is MSE, 
which corresponds to the log likelihood of the normal distribution,
\begin{equation}
\ell_2 = -n \ln \sigma  - \frac{n}{2} \ln(2\pi) - \frac{1}{2\sigma^2} \sum_{i=1}^n (y_i - \hat y_i)^2 \text{,}
\end{equation}
where $y_i$ are the observations, $\hat y_i$ are the model predictions, and $\sigma$ is standard deviation of the error.
The final term is the most important.
Stare at it for a moment and you'll recognize it as the L2 norm,
which is essentially just the MSE.
The remaining terms normalize the result, which we need in order to compare other objective functions.
If we calibrate a model to just MSE, we could drop these terms.
However, to compare different objectives, we need to keep them.

Another common objective is mean absolute error (MAE),
which corresponds to the log likelihood of the Laplace distribution,
\begin{equation}
\ell_1 = -n \ln(2b)  - \frac{1}{b} \sum_{i=1}^n | y_i - \hat y_i| \text{,}
\end{equation}
where $b$ is mean absolute error
(also known as the L1 norm).

### Changing variables
Likelihoods for a variety of other objective functions are obtained by changing variables.
For example, the mean squared log error (MSLE), which corresponds to the lognormal log likelihood $\ell_3$,
is obtained from $\ell_2$ by changing variables
\begin{equation}
\ell_3 = \ell_2(v(y)) + \ln|v'(y)| \text{,}
\end{equation}
where $v$, the natural log in this case.

Can you define more?

### Additional reading
```
@Article{Hodson_2022,
  doi = {10.48550/ARXIV.2212.06566},
  author = {Hodson, Timothy O. and Over, Thomas M. and Smith, Tyler J. and Marshall, Lucy M.},
  title = {How to select an objective function using information theory},
  publisher = {arXiv},
  year = {2022}
}
```

In [ ]:
# print one gage just to check that the data has loaded
df.loc[("01013500", ...)]
gage

Now define a function to return the log likelihood assuming the errors follow a normal distribution, which is equivalent to using MSE as an objective function.

In [ ]:
def normal_ll(y, y_hat):
    """Compute log likelihood for the normal distribution

    Parameters
    ----------
    y : array_like
        Observations.
    y_hat : array_like
        Predictions.

    Returns
    -------
        Log likelihood

    Proof
    -----
    https://www.statlect.com/probability-distributions/normal-distribution
    """
    pass


# test
normal_ll(gage["obs"], gage["NNDAR"])  # should return -121363.3792673729

Solution below.

In [ ]:
import numpy as np


def normal_ll(y, y_hat):
    """Log likelihood for the normal distribution.

    The normal distribution is the formal likelihood for the mean squared error (MSE).


    Parameters
    ----------
    y : array_like
        Observations.
    y_hat : array_like
        Predictions.
    transform : function
        Change of variable transformation.
    gradient : function
        Gradient of the transform function.

    Proof
    -----
    https://www.statlect.com/probability-distributions/normal-distribution
    """

    e = y - y_hat
    n = len(e)
    sigma = e.std()
    return  -n * np.log(sigma) - n/2 * np.log(2 * np.pi) - 1 / (2 * sigma**2) * (e**2).sum()


Now try the log likelihood of the lognormal distribution.

Hint: Ideally use a change of variable, but you can also hard code each distribution. For the lognormal, see the PDF at https://en.wikipedia.org/wiki/Log-normal_distribution

In [ ]:
def lognormal_ll(y, y_hat):
    """Compute log likelihood for the lognormal distribution.
    
    Hint: Use a change of variables

    Parameters
    ----------
    y : array_like
        Observations.
    y_hat : array_like
        Predictions.

    Returns
    -------
        Log likelihood

    """
    pass

# test
lognormal_ll(gage["obs"], gage["NNDAR"])  # should return -108803.34768683679

Solution below.

In [ ]:
def lognormal_ll(y, y_hat):
    """
    """
    log_gradient = np.sum(np.log(np.abs(1/y)))
    ll = normal_ll(y, y_hat) + log_gradient
    return ll

## More log likelihoods
To make the benchmark interesting, we'll use the change of variable technique to derive log likelihoods for several other objective functions.

In [ ]:
import numpy as np
import scipy.stats
from scipy.stats import pearsonr


def normal_ll(y, y_hat, transform=None, gradient=1):
    """Log likelihood for the normal distribution with change of variable

    The normal distribution is the formal likelihood for the mean squared error (MSE).


    Parameters
    ----------
    y : array_like
        Observations.
    y_hat : array_like
        Predictions.
    transform : function
        Change of variable transformation.
    gradient : function
        Gradient of the transform function.

    Proof
    -----
    https://www.statlect.com/probability-distributions/normal-distribution
    """
    if transform is not None:
        y = transform(y)
        y_hat = transform(y_hat)

    e = y - y_hat
    n = len(e)
    sigma = e.std()
    log_gradient = np.sum(np.log(np.abs(gradient)))
    ll = (
        -n * np.log(sigma)
        - n / 2 * np.log(2 * np.pi)
        - 1 / (2 * sigma**2) * (e**2).sum()
        + log_gradient
    )
    return ll


def laplace_ll(y, y_hat, transform=None, gradient=1):
    """Log likelihood for Laplace distribution with change of variable

    The laplace distribution is the formal likelihood for the mean absolute
    error (MAE).

    Parameters
    ----------
    y : array_like
        Observations.
    y_hat : array_like
        Predictions.
    transform : function
        Change of variable transformation.
    gradient : function
        Gradient of the transform function.
    """
    if transform is not None:
        y = transform(y)
        y_hat = transform(y_hat)

    e = (y - y_hat).abs()
    n = len(e)
    b = e.mean()
    log_gradient = np.sum(np.log(np.abs(gradient)))
    ll = -n * np.log(2 * b) - 1 / b * e.sum() + log_gradient
    return ll.sum()


def msre_ll(y, y_hat):
    """Log likelihood for mean squared square-root error

    Parameters
    ----------
    y : array_like
    y_hat : array_like
    """
    return normal_ll(
        y, y_hat, transform=lambda x: np.sqrt(x), gradient=-1 / (2 * np.sqrt(y))
    )


def mare_ll(y, y_hat):
    """Log likelihood for mean absolute square-root error

    Parameters
    ----------
    y : array_like
    y_hat : array_like
    """
    return laplace_ll(
        y, y_hat, transform=lambda x: np.sqrt(x), gradient=-1 / (2 * np.sqrt(y))
    )


def lognormal_ll(y, y_hat):
    """Lognormal log likelihood

    The lognormal distribution is the formal likelihood for the mean squared
    log error (MSLE).

    Parameters
    ----------
    y : array_like
    y_hat : array_like
    """
    return normal_ll(y, y_hat, transform=lambda x: np.log(x), gradient=1 / y)


def mspe_ll(y, y_hat):
    """Log likelhood for mean squared percentage error

    Parameters
    ----------
    y : array_like
    y_hat : array_like

    """
    return normal_ll(y, y_hat, transform=lambda x: x / y, gradient=-1 / (y**2))


def nse_ll(y, y_hat, group="gage_id"):
    """Log likelihood for normalized squared error (NSE)

    NSE is equivalent to the Nash–Sutcliffe model efficiency coefficient.

    Parameters
    ----------
    y : array_like
    y_hat : array_like
    """
    sigma_o = y.groupby("gage_id").transform(lambda x: x.std())
    return normal_ll(y, y_hat, transform=lambda x: x / sigma_o, gradient=1 / sigma_o)


def loglaplace_ll(y, y_hat):
    """Log likelihood for log Laplace distribution

    Parameters
    ----------
    y : array_like
    y_hat : array_like
    """
    return laplace_ll(y, y_hat, transform=lambda x: np.log(x), gradient=1 / y)


def uniform_ll(y, y_hat):
    """Log likelihood for uniform distribution.

    The uniform log likelihood minimizes the maximum error.

    Parameters
    ----------
    y : array_like
    y_hat : array_like
    """
    e = np.abs(y - y_hat)
    n = len(e)
    # ll = -n * np.log(e.max()-e.min()) # standard formulation
    ll = -n * np.log(e.max() - 0)
    return ll


def bernoulli_ll(y, y_hat, groupby=None):
    """TODO and use within zi_ll

    Parameters
    ----------
    y : array_like
    y_hat : array_like
    """
    pass


def zi_ll(y, y_hat, ll=normal_ll, threshold=0.01, groupby=None):
    """Zero-inflated log likelihood.

     Parameters
    ----------
    y : array_like
    y_hat : array_like
    ll : function
        Zero-inflated log likelihood
    threshold : float
        Value below which is treated as zero
    groupby : string
        Optional groupby term (testing)
    """
    y_o = y <= threshold
    y_hat_o = y_hat <= threshold

    if groupby is None:
        n1 = (y_o & y_hat_o).sum()  # correct zero-flow prediction
        n2 = (y_o ^ y_hat_o).sum()  # incorrect zero-flow prediction
    else:
        n1 = (y_o & y_hat_o).groupby(groupby).sum()  # correct zero-flow prediction
        n2 = (y_o ^ y_hat_o).groupby(groupby).sum()  # incorrect zero-flow prediction

    n3 = ~y_o & ~y_hat_o  # correct flow predictions

    # fraction of correctly predicted zero flows
    rho = np.where((n1 + n2) == 0, 0, n1 / (n1 + n2))
    n_rho = 1 - rho

    # n1 * np.log(rho) + n2 * np.log(1-rho)
    ll_zero = n1[rho != 0] * np.log(rho[rho != 0]) + n2[n_rho != 0] * np.log(
        n_rho[n_rho != 0]
    )

    return ll_zero.sum() + ll(y[n3], y_hat[n3])


def zilognormal_ll(y, y_hat):
    """Log likelihood for zero-inflated lognormal.

    Parameters
    ----------
    y : array_like
    y_hat : array_like
    """

    return zi_ll(y, y_hat, ll=lognormal_ll, threshold=0.01)


def ziloglaplace_ll(y, y_hat):
    """Log likelihood for zero-inflated laplace.

    Parameters
    ----------
    y : array_like
    y_hat : array_like
    """
    return zi_ll(y, y_hat, ll=loglaplace_ll, threshold=0.01)

## The experiment
Like any model, to evaluate competing objective functions, we compare their log likelihoods.


In [ ]:
# step 1: create a table of objective functions
objectives = {
    "U": {"name": "uniformly distributed error", "f": uniform_ll},
    "MSE": {"name": "mean squared error", "f": normal_ll},
    "NSE": {"name": "normalized squared error", "f": nse_ll},
    "MAE": {"name": "mean absolute error", "f": laplace_ll},
    "MSPE": {"name": "mean squared percent error", "f": mspe_ll},
    "MSLE": {"name": "mean squared log error*", "f": lognormal_ll},
    "MALE": {"name": "mean absolute log error*", "f": loglaplace_ll},
    "ZMSLE": {"name": "zero-inflated MSLE", "f": zilognormal_ll},
    "ZMALE": {"name": "zero-inflated MALE", "f": ziloglaplace_ll},
    "MARE": {"name": "mean absolute square root error", "f": mare_ll},
}

obj_df = pd.DataFrame.from_dict(objectives, orient="index")

# step 2: compute the information in each objective function
for index, row in obj_df.iterrows():
    # nats is the negative log likelihood or the info in the error
    obj_df.loc[index, "ll"] = row.f(df.obs, df.NNDAR)

obj_df = obj_df[["name", "ll"]]

obj_df

But raw log likelihoods can be difficult to digest, so we typically rescale them.

## Akaike weights

In practice, we aren't intersted in the magnitude of the log likelihood, only their differences.
One approach is to represent each by its Akaike weights, which represents the probability that each objective is the correct on.

In [ ]:
def compute_weights(series, base=np.e):
    """Compute posterior weights

    Parameters
    ----------
    series : array_like
        Log likelihoods
    base: float
        Base of the logarithm used to compute log likelihood
    """
    s = base**series
    return s / s.sum()

## Bits

We can also convert the log likelihood to bits. Their magnitude is still meaningless, but at least the scale is more digestable.

In [ ]:
def compute_info(ll, n, base=2):
    """Convert a log likelihood to bits

    Parameters
    ----------
    ll : array_like
        Log likelihoods
    n :
    base: float
        Base of the logarithm; 2 = bits
    """
    return -ll / n / np.log(base)

Now we're ready to generate some results.

In [ ]:
# step 1: create a table of objective functions
objectives = {
    "U": {"name": "uniformly distributed error", "f": uniform_ll},
    "MSE": {"name": "mean squared error", "f": normal_ll},
    "NSE": {"name": "normalized squared error", "f": nse_ll},
    "MAE": {"name": "mean absolute error", "f": laplace_ll},
    "MSPE": {"name": "mean squared percent error", "f": mspe_ll},
    "MSLE": {"name": "mean squared log error*", "f": lognormal_ll},
    "MALE": {"name": "mean absolute log error*", "f": loglaplace_ll},
    "ZMSLE": {"name": "zero-inflated MSLE", "f": zilognormal_ll},
    "ZMALE": {"name": "zero-inflated MALE", "f": ziloglaplace_ll},
    "MARE": {"name": "mean absolute square root error", "f": mare_ll},
}

obj_df = pd.DataFrame.from_dict(objectives, orient='index')

# step 2: compute the information in each objective function
for index, row in obj_df.iterrows():
    # nats is the negative log likelihood or the info in the error
    #obj_df.loc[index, 'bits'] = - row.f(df.obs, df.NNDAR)/len(df)/np.log(2)
    ll = row.f(df.obs, df.NNDAR)
    obj_df.loc[index, 'bits'] = compute_info(ll, len(df))
    
# step 3: compute weights
obj_df['weight'] = compute_weights(-obj_df.bits, base=2)

# step 4: format output table

table = obj_df[['name','bits','weight']].sort_values('weight').round(2)#.rename(columns=names)

table['rank'] = len(table) - np.argsort(table['weight'])

table

In [ ]:
# compute noise in MSE
nse_bits = table.loc["NSE", "bits"]
zmale_bits = table.loc["ZMALE", "bits"]
nse_noise = round(100 * (nse_bits - zmale_bits) / nse_bits)
print(
    f"NSE is at least {nse_noise} percent noise"
)